## Worm-Like Chain (WLC) Simulation Tutorial for UAMMD-structured

## Introduction
This notebook demonstrates how to create and run a simulation of polymer chains using the Worm-Like Chain (WLC) model in UAMMD-structured. The WLC model is commonly used to describe semi-flexible polymers. We'll walk through the process step-by-step, explaining each part of the simulation setup in detail.

## Setup

First, let's import the necessary libraries:

In [ ]:
import os
import numpy as np
import pyUAMMD

def make_helix(N, radius=1.0, pitch=3.0, loops=3):
    """
    Genera las coordenadas de una hélice periódica en 3D.

    Parámetros:
    -----------
    N : int
        Número total de partículas en la hélice.
    radius : float
        radius de la hélice.
    pitch : float
        pitch (altura por vuelta).
    loops : int
        Número de loops completas antes de repetirse.

    Retorna:
    --------
    coords : np.ndarray de forma (N,3)
        Coordenadas [x,y,z] de cada partícula.
    """
    assert N > 0, "El número de partículas debe ser positivo."
    assert int(N/loops) == N/loops, "El número de partículas debe ser múltiplo del número de loops."
    # Separación angular para que cierre tras "loops"
    delta_theta = 2 * np.pi * loops / N
    
    # Ángulos
    thetas = np.arange(N) * delta_theta
    
    # Coordenadas
    x = radius * np.cos(thetas)
    y = radius * np.sin(thetas)
    z = (pitch / (2*np.pi)) * thetas
    
    return np.column_stack((x, y, z))


def dihedral(p0,p1,p2,p3):
    b0 = p1 - p0
    b1 = p2 - p1
    b2 = p3 - p2

    n1 = np.cross(b0, b1)
    n2 = np.cross(b1, b2)

    n1_norm = n1 / np.linalg.norm(n1)
    n2_norm = n2 / np.linalg.norm(n2)
    m1 = np.cross(n1_norm, b1/np.linalg.norm(b1))
    x = np.dot(n1_norm, n2_norm)
    y = np.dot(m1, n2_norm)
    angle = np.atan2(y, x)

    return angle

## Simulation Parameters

Here we define the key parameters for our WLC simulation:

In [2]:
# Number of beads per loop
nBeadsPerLoop = 15
loops = 3
nBeads = nBeadsPerLoop * loops
pitch = 8

# Bead diameter (and bond length)
sigma = 1.0
helix_coords = make_helix(N=nBeads, radius=2*sigma, pitch=pitch, loops=loops)
#Center the helix coords:
helix_coords -= np.mean(helix_coords, axis=0)

# Calculate box size (add some extra space)
L = 4*np.max(np.abs(helix_coords)) + 2.0*sigma

# Simulation parameters
timeStep = 0.01
frictionConstant = 3*np.pi

# Total number of simulation steps
nSteps = 30000

# Frequency of information output
nStepsInfo = 3000

# Frequency of trajectory output
nStepsOutput = 100

# Force constants for bonds and angles
T = 0.01
Kb = 100.0  # Bond strength
Ka = 100.0  # Angle strength (bending rigidity)
Kc = 100.0    # Dihedral strength

print("Creating a WLC simulation with:")
print(f" - Number of beads: {nBeads}")
print(f" - Bead diameter: {sigma}")
print(f" - Box size: {L}")
print(f" - Time step: {timeStep}")
print(f" - Friction constant: {frictionConstant}")
print(f" - Total number of steps: {nSteps}")
print(f" - Bond strength (Kb): {Kb}")
print(f" - Bending rigidity (Ka): {Ka}")
print(f" - Output information every {nStepsInfo} steps")
print(f" - Output trajectory every {nStepsOutput} steps")

Creating a WLC simulation with:
 - Number of beads: 45
 - Bead diameter: 1.0
 - Box size: 48.93333333333333
 - Time step: 0.01
 - Friction constant: 9.42477796076938
 - Total number of steps: 30000
 - Bond strength (Kb): 100.0
 - Bending rigidity (Ka): 100.0
 - Output information every 3000 steps
 - Output trajectory every 100 steps



## Creating the Simulation

Now, let's create our simulation object and set up its various components:

In [3]:
# Initialize the simulation object
simulation = pyUAMMD.simulation()

# Set up the system information
simulation["system"] = {
    "info": {
        "type": ["Simulation", "Information"],
        "parameters": {"name": "WormLikeChain"}
    }
}

# Define global parameters
simulation["global"] = {
    # Set the unit system (in this case, we're using reduced units)
    "units": {"type": ["Units", "None"]},

    # Define particle types
    "types": {
        "type": ["Types", "Basic"],
        "labels": ["name", "mass", "radius", "charge"],
        "data": [["A", 1.0, sigma/2.0, 0.0]]
    },

    # Set the ensemble (NVT: constant Number of particles, Volume, and Temperature)
    "ensemble": {
        "type": ["Ensemble", "NVT"],
        "labels": ["box", "temperature"],
        "data": [[[L, L, L], T]]
    }
}

# Set up the integrator (Langevin dynamics)
simulation["integrator"] = {
    "bbk": {
        "type": ["Langevin", "BBK"],
        "parameters": {
            "timeStep": timeStep,
            "frictionConstant": frictionConstant
        }
    },
    # Define the integration schedule
    "schedule": {
        "type": ["Schedule", "Integrator"],
        "labels": ["order", "integrator", "steps"],
        "data": [[1, "bbk", nSteps]]
    }
}

## Initialize Particle Positions and Topology

We'll place the polymer chains in the simulation box and set up the topology:

In [4]:
simulation["state"] = {
    "labels": ["id", "position"],
    "data": []
}

simulation["topology"] = {
    "structure": {
        "labels": ["id", "type", "modelId"],
        "data": []
    }
}

particleId = 0

for j in range(int(nBeads)):
    # Place beads along the z-axis, centered at the origin
    simulation["state"]["data"].append([particleId, [helix_coords[j, 0], helix_coords[j, 1], helix_coords[j, 2]]])
    simulation["topology"]["structure"]["data"].append([particleId, "A", j])
    particleId += 1

## Define Force Field

Set up the bonded interactions for our WLC model:

In [5]:
simulation["topology"]["forceField"] = {}

# Set up harmonic bonds
simulation["topology"]["forceField"]["bonds"] = {
    "type": ["Bond2", "Harmonic"],
    "parameters": {},
    "labels": ["id_i", "id_j", "K", "r0"],
    "data": []
}

particleId = 0
r0 = np.linalg.norm(np.diff(helix_coords, axis=0), axis=1)  # N-1 valores
for j in range(int(nBeads) - 1):
    simulation["topology"]["forceField"]["bonds"]["data"].append([particleId, particleId + 1, Kb, r0[j]])
    particleId += 1

particleId = 0
r_pitch = np.linalg.norm((helix_coords[0]-helix_coords[nBeadsPerLoop]), axis=0)
for j in range(int(nBeads) - nBeadsPerLoop):
    simulation["topology"]["forceField"]["bonds"]["data"].append([particleId, particleId + nBeadsPerLoop, Kb, r_pitch])
    particleId += 1

simulation["topology"]["forceField"]["angles"] = {
    "type": ["Bond3", "KratkyPorod"],
    "parameters": {},
    "labels": ["id_i", "id_j", "id_k", "K"],
    "data": []
}

particleId = 0
for j in range(int(nBeads) - 2):
    simulation["topology"]["forceField"]["angles"]["data"].append([particleId, particleId + 1, particleId + 2, Ka])
    particleId += 1

particleId = 0
for j in range(int(nBeads) - 2*nBeadsPerLoop):
    simulation["topology"]["forceField"]["angles"]["data"].append([particleId, particleId + nBeadsPerLoop, particleId + 2*nBeadsPerLoop, Ka])
    particleId += 1

# # Set up angle interactions (Harmonic potential for bending rigidity)
dihedral_angle = dihedral(helix_coords[0], helix_coords[1], helix_coords[2], helix_coords[3])
print(dihedral_angle)
simulation["topology"]["forceField"]["dihedrals"] = {
    "type": ["Bond4", "DihedralCommon_n_K_phi0"],
    "parameters": {"n": 1,
                   "K": Kc*0,
                   "phi0": dihedral_angle},
    "labels": ["id_i", "id_j", "id_k", "id_l"],
    "data": []
}

particleId = 0
for j in range(int(nBeads) - 3):
    simulation["topology"]["forceField"]["dihedrals"]["data"].append([particleId, particleId + 1, particleId + 2, particleId + 3])
    particleId += 1


-0.22848891856269246


## Configure Simulation Steps

Define what operations to perform during the simulation:

In [6]:
simulation["simulationStep"] = {
    # Output simulation information periodically
    "info": {
        "type": ["UtilsStep", "InfoStep"],
        "parameters": {"intervalStep": nStepsInfo}
    },
    # Save trajectory data periodically
    "output": {
        "type": ["WriteStep", "WriteStep"],
        "parameters": {
            "intervalStep": nStepsOutput,
            "outputFilePath": "output",
            "outputFormat": "sp"
        }
    }
}

## Write the Simulation File

Finally, let's write our simulation to a JSON file:

In [7]:
print()
print("Writing simulation file...")
simulation.write("simulation.json")
print("Simulation file created successfully!")
print()

08/09/2025 12:54:49 - pyUAMMD - WARNING - Writing with legacy 



Writing simulation file...
Simulation file created successfully!



## Running the Simulation

To run the simulation, you would typically use the UAMMD-structured executable with the generated JSON file.

In [8]:
print()
print("You can run the code now using: UAMMDlauncher simulation.json, or with simulation.run()")
# simulation.run()


You can run the code now using: UAMMDlauncher simulation.json, or with simulation.run()


In [9]:
# # Set up harmonic bonds
# simulation["topology"]["forceField"]["surface"] = {
#     "type":["Surface","SurfaceWCAType1"],
#     "parameters":{
#         "surfacePosition": -L/2 + 5.0*sigma,
#     },
#     "labels":["name", "epsilon", "sigma"],
#     "data":[
#         ["A", 1.0, 1.0]
#   ]
# }
simulation["integrator"]["bbk"] = {
    "type":["FluctuatingHydrodynamics","IncompressibleInertialCoupling"],
    "parameters":{
    "timeStep": timeStep,
    "viscosity": 1.0,
    "density": 1.0,
    "hydrodynamicRadius": 0.5*sigma,
    "sumThermalDrift": False,
    "removeTotalMomentum": True
    }
}
# Set up harmonic bonds
simulation["topology"]["forceField"]["gravity"] = {
    "type": ["External", "ConstantForce"],
    "parameters": {"constantForce": [0.0, 0.0, -1]}
}

simulation["simulationStep"] = {
    # Output simulation information periodically
    "info": {
        "type": ["UtilsStep", "InfoStep"],
        "parameters": {"intervalStep": nStepsInfo}
    },
    # Save trajectory data periodically
    "output": {
        "type": ["WriteStep", "WriteStep"],
        "parameters": {
            "intervalStep": nStepsOutput,
            "outputFilePath": "gravity_output",
            "outputFormat": "sp",
            "pbc": False
        }
    }
}

In [10]:
simulation.write("simulation_gravity.json")
simulation.run()

08/09/2025 12:54:49 - pyUAMMD - WARNING - Writing with legacy 
[WARNING] UAMMD-structured Python wrapper is not compatible with UAMMD-structured self restarting mechanism
[MESSAGE] ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
[MESSAGE] ╻ ╻┏━┓┏┳┓┏┳┓╺┳┓
[MESSAGE] ┃ ┃┣━┫┃┃┃┃┃┃ ┃┃ Version: 2.5
[MESSAGE] ┗━┛╹ ╹╹ ╹╹ ╹╺┻┛
[MESSAGE] Compiled at: May 30 2025 09:48:24
[MESSAGE] Compiled in double precision mode
[MESSAGE] Computation started at Mon Sep  8 12:54:49 2025

[MESSAGE] [System] CUDA initialized
[MESSAGE] [System] Using device: NVIDIA GeForce RTX 4070 Ti with id: 0
[MESSAGE] [System] Compute capability of the device: 8.9
[MESSAGE] ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ 
[MESSAGE] [ExtendedSystem] (system) Name: WormLikeChain
[MESSAGE] [ExtendedSystem] (system) Seed: 1757328890124187674
[MESSAGE] [GlobalDataBase] Fundamental not specified, using default fundamental, "Time"
[WARNING] [Time] No timeStep specified, using 0.0 as default.
[MESSAGE] [Basic] 

## Conclusion

This tutorial demonstrated how to set up and prepare a simulation of Worm-Like Chains using UAMMD-structured. We covered:
1. Defining simulation parameters, including polymer-specific parameters
3. Creating the simulation object
4. Setting up the system, global parameters, and integrator
5. Initializing particle positions and topology for multiple polymer chains
6. Configuring the simulation topology and force field, including bonded and angle interactions
7. Setting up simulation steps for output
8. Writing the simulation file
9. Running the simulation

Next steps could include:
- Analyzing the output trajectory
- Visualizing the polymer chains
- Calculating polymer properties such as end-to-end distance or radius of gyration
- Modifying the simulation parameters to explore different polymer lengths, stiffnesses, or environmental conditions